In [1]:
import re
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

# Extracting data

In [2]:
%%time
df = pd.read_csv('chess_games.csv', nrows=500000,
                usecols=['Event', 'Result', 'UTCDate', 'Opening', 'Termination', 'AN']) # chunksize=100000, index_col=''
print(f"{len(df)} rows was extracted")

500000 rows was extracted
CPU times: total: 4.44 s
Wall time: 4.55 s


In [3]:
df.head(3)

,Event,Result,UTCDate,Opening,Termination,AN
0,Classical,1-0,2016.06.30,Slav Defense,Time forfeit,1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1,Blitz,0-1,2016.06.30,King's Pawn Opening: 2.b3,Normal,1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2,Blitz tournament,1-0,2016.06.30,Scandinavian Defense: Mieses-Kotroc Variation,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....


# Transforming data

## Filter dataframe

### Get rid of ambiguous match results and matches that was abandoned or have rules infractions

In [4]:
filt = (df['Result'] != '*') & (df['Termination'] != 'Abandoned') & (df['Termination'] != 'Rules infraction')
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

1057 rows was removed
498943 rows left


### Remove short matches (~less than 10 moves)

In [5]:
filt = df["AN"].apply(len) > 50
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

5689 rows was removed
493254 rows left


### Get rid of unstructured string values in "AN" column

In [6]:
filt = ~df['AN'].str.contains('\[%eval')
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

63866 rows was removed
429388 rows left


### Remove unpopular openings (less than 1500 matches)

In [86]:
# value_counts = df.stack().value_counts() # Entire DataFrame 
# to_remove = value_counts[value_counts <= 1500].index
# df.replace(to_remove, np.nan, inplace=True)

In [87]:
vc = df['Opening'].value_counts()
vals_to_remove = vc[vc < 1500].index.values
df['Opening'].loc[df['Opening'].isin(vals_to_remove)] = '111'
filt = df['Opening'] != '111'
print(f"{len(df)-len(df[filt])} rows was removed")
df = df[filt]
print(f"{len(df)} rows left")

255831 rows was removed
173557 rows left


## Expand and rearrange main dataframe

### Reset index

In [7]:
df = df.reset_index(drop=True)

### Add Tournament column

In [8]:
df['Tournament'] = df['Event'].str.contains('tournament')

### Rearrange column "Event"

In [9]:
df['Event'] = df['Event'].map({' Classical ': 'Classical',
                               ' Blitz ': 'Blitz',
                               ' Blitz tournament ': 'Blitz',
                               ' Correspondence ': 'Classical',
                               ' Classical tournament ': 'Classical',
                               ' Bullet tournament ': 'Bullet',
                               ' Bullet ': 'Bullet',
                               'Blitz tournament ': 'Blitz',
                               'Bullet ': 'Bullet',
                               'Classical ': 'Classical',
                               'Blitz ': 'Blitz'
                               })

### Change columns datatype

In [10]:
df['Event'] = df['Event'].astype('category')
df['Result'] = df['Result'].astype('category')
df['Termination'] = df['Termination'].astype('category')
df['UTCDate'] = pd.to_datetime(df['UTCDate'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429388 entries, 0 to 429387
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Event        429388 non-null  category      
 1   Result       429388 non-null  category      
 2   UTCDate      429388 non-null  datetime64[ns]
 3   Opening      429388 non-null  object        
 4   Termination  429388 non-null  category      
 5   AN           429388 non-null  object        
 6   Tournament   429388 non-null  bool          
dtypes: bool(1), category(3), datetime64[ns](1), object(2)
memory usage: 11.5+ MB


## Create moves dataframe

### Create moves_total from AN column

In [12]:
def delete_result_from_an(move: str) -> str:
    '''Delete result mark in the last move.'''
    move = re.sub(' 1-0',     '', move)
    move = re.sub(' 0-1',     '', move)
    move = re.sub(' 1/2-1/2', '', move)
    move = re.sub(' \*',      '', move)
    return move

In [13]:
def populate_moves_total(s: str, ind: int) -> None:
    '''
    Split each move (string) into a tuple.
    Tuple format: (match_id, move_num, player, move)
    Match_id is foreign key of the main dataframe (index column).
    Player is categorical column with values 'white' or 'black'.
    '''
    player_cnt = 0
    moves_list = s.split(' ')
    for i, move in enumerate(moves_list, start=1):
        player = 'black' if i % 2 == 0 else 'white'
        tuple_move = tuple([ind] + [i] + [player] + [move])
        moves_total.append(tuple_move)

In [14]:
def create_moves_total(row) -> None:
    s = row["AN"]
    sep_num = 1
    while True:
        if str(sep_num)+'. ' in s:
            s = re.sub(str(sep_num)+'\. ', '', s, 1) # replace the first occurrence
            sep_num+=1
        else:
            s = delete_result_from_an(s)
            break

    populate_moves_total(s, row.name)

In [15]:
%%time
%%capture
moves_total: list[tuple] = [] # template for data frame
df.apply(create_moves_total, axis = 1)

CPU times: total: 1min 22s
Wall time: 1min 32s


In [16]:
len(moves_total)

29424621

### Convert moves_total to df_moves

In [17]:
%%time
df_moves = pd.DataFrame(moves_total, columns=['match_id', 'move_num', 'player', 'move'])

CPU times: total: 43.5 s
Wall time: 1min 53s


In [18]:
df_moves.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29424621 entries, 0 to 29424620
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   match_id  int64 
 1   move_num  int64 
 2   player    object
 3   move      object
dtypes: int64(2), object(2)
memory usage: 898.0+ MB


In [19]:
df_moves['player'] = df_moves['player'].astype('category')
df_moves.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29424621 entries, 0 to 29424620
Data columns (total 4 columns):
 #   Column    Dtype   
---  ------    -----   
 0   match_id  int64   
 1   move_num  int64   
 2   player    category
 3   move      object  
dtypes: category(1), int64(2), object(1)
memory usage: 701.5+ MB


### Drop AN column in main dataframe

In [20]:
df = df.drop('AN', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429388 entries, 0 to 429387
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Event        429388 non-null  category      
 1   Result       429388 non-null  category      
 2   UTCDate      429388 non-null  datetime64[ns]
 3   Opening      429388 non-null  object        
 4   Termination  429388 non-null  category      
 5   Tournament   429388 non-null  bool          
dtypes: bool(1), category(3), datetime64[ns](1), object(1)
memory usage: 8.2+ MB


## Populate df_moves

### Make sure that all move have correct [Algebraic notation](https://en.wikipedia.org/wiki/Algebraic_notation_(chess))

#### Format:
piece_name, disambiguating (optional), capture (optional), square_name, pawn_promotion (optional), check or checkmate (optional) \
kingside or queenside castling, check or checkmate (optional)

In [69]:
# filt = ~(df_moves['move'].str.match(r'(^|[KQRBN])[a-h]?[1-8]?([a-h]?x)?[a-h][1-8](=[QRBN])?[\+#]?$') == True) \
#      & ~(df_moves['move'].str.match(r'^(O-O|O-O-O)[\+#]?$')== True)
# df_moves[filt]

In [66]:
filt = ~(df_moves['move'].str.match(r'(^|[KQRBN])[a-h]?[1-8]?x?[a-h][1-8](=[QRBN])?[\+#]?$') == True) \
     & ~(df_moves['move'].str.match(r'^(O-O|O-O-O)[\+#]?$')== True)
df_moves[filt]

,match_id,move_num,player,move


In [22]:
m = re.match(r'(?P<piece_name>^|[KQRBN])[a-h]?[1-8]?(?P<capture>[a-h]?x)?[a-h][1-8](?P<promotion>=[QRBN])?(?P<position>[\+#])?$', 'Qh4xe1+')
m.groupdict()

{'piece_name': 'Q', 'capture': 'x', 'promotion': None, 'position': '+'}

In [23]:
df_test = df_moves.copy()

### Add castling column

In [25]:
%%time
castling_conditions = [
    df_test['move'] == 'O-O',
    df_test['move'] == 'O-O+',
    df_test['move'] == 'O-O#',
    df_test['move'] == 'O-O-O',
    df_test['move'] == 'O-O-O+',
    df_test['move'] == 'O-O-O#'
    # df_test['move'].str.match(r'^(O-O)[\+#]?$') == True,
    # df_test['move'].str.match(r'^(O-O-O)[\+#]?$') == True
]

castling_outputs = [
    'kingside', 'kingside', 'kingside', 'queenside', 'queenside', 'queenside'
    # 'kingside', 'queenside'
]

castling = np.select(castling_conditions, castling_outputs, None)
df_test['castling'] = pd.Series(castling)

CPU times: total: 11.9 s
Wall time: 12.1 s


In [43]:
df_test['castling'] = df_test['castling'].astype('category')

### Add piece name column

In [37]:
%%time
piece_conditions = [
    df_test['move'].str.contains('O-O', regex=False) == True, # castling is considered as a king move
    df_test['move'].str.match(r'^K') == True,
    df_test['move'].str.match(r'^Q') == True,
    df_test['move'].str.match(r'^R') == True,
    df_test['move'].str.match(r'^B') == True,
    df_test['move'].str.match(r'^N') == True,
    df_test['move'].str.match(r'^[a-h]') == True # pawn don't have piece name in AN
]

piece_outputs = [
    'king', 'king', 'queen', 'rook', 'bishop', 'knight', 'pawn'
]

piece_name = np.select(piece_conditions, piece_outputs, None)
df_test['piece_name'] = pd.Series(piece_name)

CPU times: total: 2min 9s
Wall time: 2min 13s


In [50]:
df_test['piece_name'] = df_test['piece_name'].astype('category')

### Add capture column

In [73]:
df_test['capture'] = df_test['move'].str.contains('x') == True

### Add pawn promotion column

In [80]:
%%time
pawn_promotion_conditions = [
    df_test['move'].str.contains('=Q') == True,
    df_test['move'].str.contains('=R') == True,
    df_test['move'].str.contains('=B') == True,
    df_test['move'].str.contains('=N') == True
]

pawn_promotion_outputs = [
    'queen', 'rook', 'bishop', 'knight'
]

pawn_promotion = np.select(pawn_promotion_conditions, pawn_promotion_outputs, None)
df_test['pawn_promotion'] = pd.Series(pawn_promotion)

CPU times: total: 1min 1s
Wall time: 1min 1s


In [98]:
df_test['pawn_promotion'] = df_test['pawn_promotion'].astype('category')

### Add position column (check, checkmate)

In [100]:
%%time
position_conditions = [
    df_test['move'].str.contains(r'\+$') == True,
    df_test['move'].str.contains(r'#$') == True,
]

position_outputs = [
    'check', 'checkmate'
]

position = np.select(position_conditions, position_outputs, None)
df_test['position'] = pd.Series(position)

CPU times: total: 32.6 s
Wall time: 34.7 s


In [112]:
df_test['position'] = df_test['position'].astype('category')

### Dataframe testing

In [114]:
df_test.head()

,match_id,move_num,player,move,castling,piece_name,capture,pawn_promotion,position
0,0,1,white,d4,NaN,pawn,False,NaN,NaN
1,0,2,black,d5,NaN,pawn,False,NaN,NaN
2,0,3,white,c4,NaN,pawn,False,NaN,NaN
3,0,4,black,c6,NaN,pawn,False,NaN,NaN
4,0,5,white,e3,NaN,pawn,False,NaN,NaN


In [109]:
df_test['position'].isna().sum()

27204498

In [110]:
df_test['position'].value_counts()

position
check        2108092
checkmate     112031
Name: count, dtype: int64

In [113]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29424621 entries, 0 to 29424620
Data columns (total 9 columns):
 #   Column          Dtype   
---  ------          -----   
 0   match_id        int64   
 1   move_num        int64   
 2   player          category
 3   move            object  
 4   castling        category
 5   piece_name      category
 6   capture         bool    
 7   pawn_promotion  category
 8   position        category
dtypes: bool(1), category(5), int64(2), object(1)
memory usage: 841.8+ MB


### OLD

In [115]:
def extract_regex(row: str):
    column_piece_name = []
    m = re.match(r'(?P<piece_name>^|[KQRBN])[a-h]?[1-8]?(?P<capture>[a-h]?x)?[a-h][1-8](?P<promotion>=[QRBN])?(?P<position>[\+#])?$', row)
    piece_name = m['piece_name']
    new_column.append(piece_name)
    capture = m['capture']
    promotion = m['promotion']
    position = m['position']
    return pd.Series([m.groupdict()])

In [121]:
extract_regex('e8=Q+')[0]['position']

'+'

In [112]:
%%time
# df_captured = df_test['move'].apply(lambda x: extract_regex(x, full_regex))
df_captured = df_test['move'].apply(extract_regex)

,match_id,move_num,player,move,piece_name
0,0,1,white,e4,None
1,0,2,black,d5,None
2,0,3,white,exd5,None
3,0,4,black,Qxd5,None
4,0,5,white,Nf3,None
...,...,...,...,...,...
11971285,173556,89,white,Bg3,None
11971286,173556,90,black,Kxe6,None
11971287,173556,91,white,Kc3,None
11971288,173556,92,black,Kd5,None


In [29]:
# column_piece_name = []
# for move in df_test['move']:
#     #print(move)
#     m = re.match(r'(?P<piece_name>^|[KQRBN])[a-h]?[1-8]?(?P<capture>[a-h]?x)?[a-h][1-8](?P<promotion>=[QRBN])?(?P<position>[\+#])?$', values)
#     #print(m)
#     m.groupdict()
#     #print(m.groupdict())
#     if m['piece_name']:
#         piece_name = m['piece_name']
#     else:
#         piece_name = 'Pawn'
#     column_piece_name.append(piece_name)

# df_test['piece_name'] = column_piece_name

In [30]:
# def castling(row: str):
#     if row['move'] == 'O-O': # re.match(r'^(O-O)[\+#]?$', row) == True:
#         return 'kingside'
#     elif row['move'] == 'O-O-O': # re.match(r'^(O-O-O)[\+#]?$', row) == True:
#         return 'queenside'
#     else:
#         return None

In [31]:
# %%time
# # df_test['castling'] = df_test['move'].apply(castling)
# df_test['castling'] = df_test.apply(castling, axis=1)

In [ ]:
# %%time
# for i, row in df.iterrows():
#     df.at[i, 'Event'] = df.at[i, 'Event']

In [ ]:
# %%time
# for i in df.index:
#     df['Event'].iloc[i] = df['Event'].iloc[i]